In [1]:
# for Mathematical and Dataframe Operations
import numpy as np
import pandas as pd

# for Data Visualizations
import seaborn as sns
import matplotlib.pyplot as plt

### Reading the Dataset

In [2]:
# reading the data set
data = pd.read_csv('data.csv')

# Print the Shape
data.shape

(1143, 11)

In [3]:
# lets check the head of the dataset
data.head()

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
0,708746,916,103916,30-34,M,15,7350,1,1.43,2,1
1,708749,916,103917,30-34,M,16,17861,2,1.82,2,0
2,708771,916,103920,30-34,M,20,693,0,0.00,1,0
3,708815,916,103928,30-34,M,28,4259,1,1.25,1,0
4,708818,916,103928,30-34,M,28,4133,1,1.29,1,1


In [201]:
## lets analyze the Campaigns

print("Number of Unique Ads :", data['ad_id'].nunique())
print("Number of Campaigns :", data['xyz_campaign_id'].nunique())
print("Number of Facebook Campaigns :", data['fb_campaign_id'].nunique())
print("Number of Interest Groups :", data['interest'].nunique())
print("Number of Age Groups :", data['age'].nunique())

Number of Unique Ads : 1143
Number of Campaigns : 3
Number of Facebook Campaigns : 691
Number of Interest Groups : 40
Number of Age Groups : 4


In [202]:
# as we can see that every ad is having an Unique ID, so we will remove that column.
# again, Facebook campaign has no relation with any conversion, so, let's remove this column also.

data = data.drop(['ad_id', 'fb_campaign_id'], axis = 1)

# now lets check the column names again to make sure above columns are removed
data.columns

Index(['xyz_campaign_id', 'age', 'gender', 'interest', 'Impressions', 'Clicks',
       'Spent', 'Total_Conversion', 'Approved_Conversion'],
      dtype='object')

### Feature Engineering

* ```Click-through-rate (CTR)```: This is the percentage of how many of our impressions became clicks. A high CTR is often seen as a sign of good creative being presented to a relevant audience. A low click through rate is suggestive of less-than-engaging adverts (design and / or messaging) and / or presentation of adverts to an inappropriate audience. What is seen as a good CTR will depend on the type of advert (website banner, Google Shopping ad, search network test ad etc.) and can vary across sectors, but 2% would be a reasonable benchmark.

--------------
* ```Conversion Rate (CR)```: This is the percentage of clicks that result in a 'conversion'. What a conversion is will be determined by the objectives of the campaign. It could be a sale, someone completing a contact form on a landing page, downloading an e-book, watching a video, or simply spending more than a particular amount of time or viewing over a target number of pages on a website.

-------------
* ```Cost Per Click (CPC)```: Self-explanatory this one: how much (on average) did each click cost. While it can often be seen as desirable to reduce the cost per click, the CPC needs to be considered along with other variables. For example, a campaign with an average CPC of £0.5 and a CR of 5% is likely achieving more with its budget than one with a CPC of £0.2 and a CR of 1% (assuming the conversion value is the same.

-----------------
* ```Cost Per Conversion```: Another simple metric, this figure is often more relevant than the CPC, as it combines the CPC and CR metrics, giving us an easy way to quickly get a feel for campaign effectiveness.

In [206]:
# Performing Feature Engineering to understand the Campaigns in Detail

# lets create a Column to understand CTR
data['ClickThroughRate'] = ((data['Clicks']/data['Impressions'])*100)

# lets create a column to understand CPC
data['CostPerClick'] = (data['Spent']/data['Clicks'])

# lets create a column to understand Conversion Rate
data['ConversionRate'] = (data['Approved_Conversion']/data['Total_Conversion'])

# lets create a column to understand the Cost per Conversion
data['CostPerConversion'] = (data['Spent']/data['Approved_Conversion']) 

* ```Conversion Value```: A conversion value is a numerical value that you assign to specific conversions in order to represent their impact to your business. The major benefit to assigning conversion values is to help you track, optimize, and report on your return on ad spend (ROAS).

----------------
* ```ROAS```: ROAS stands for return on ad spend—a marketing metric that measures the amount of revenue your business earns for each dollar it spends on advertising. For all intents and purposes, ROAS is practically the same as another metric you're probably familiar with: return on investment, or ROI.

-----------------
* ```Cost Per Mille```: This number is the cost of one thousand impressions. If your objective is ad exposure to increase brand awareness, this might be an important KPI for you to measure.

**We don't have the actual numbers to understand the Profit of after Conversion, but for the purposes of Understanding, let's assume that an Approved conversion is worth 100 dollars.** 

In [207]:
# lets add More Features to Understand the Return on Investment also known as ROAS(Return on Ad Spend)

# lets calculate the Conversion Value, Let's Assume that the Value of Sales if 100 Dollars
data['ConversionValue'] = data['Approved_Conversion']*100

# lets calculate the ROAS Which is the Target Variable for us
data['ROAS'] = round(data['ConversionValue']/data['Spent'], 2)

# lets Calculate CPM to understand the Brand Awareness from Campaigns
data['CostPerMille'] = round((data['Spent']/data['Impressions'])*1000, 2)

In [209]:
# Lets remove all the records where we have any Nan, value or Infinity Value
data = data.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

# lets check the shape of the data after remving unnecessary records
print("Shape of the Data After Removing Nans and Infs :", data.shape)

Shape of the Data After Removing Nans and Infs : (513, 16)


In [210]:
# lets Compare the Campaign based on the New Features Created

data[['xyz_campaign_id','ClickThroughRate','CostPerClick','ConversionRate',
     'ConversionValue','CostPerMille', 'ROAS']].groupby(['xyz_campaign_id'],
         as_index = False).agg('mean').rename(columns = {'xyz_campaign_id': 'Campaign',
                             'ClickThroughRate': 'Average CTR',
                             'CostPerClick': 'Average Cost/Click', 
                        'ConversionRate':'Average ConversionRate'}).style.background_gradient(cmap = 'Wistia')

,Campaign,Average CTR,Average Cost/Click,Average ConversionRate,ConversionValue,CostPerMille,ROAS
0,campaign_a,0.027162,1.339464,0.906250,100.000000,0.371250,42.500000
1,campaign_b,0.026293,1.372928,0.899851,105.357143,0.359732,47.035089
2,campaign_c,0.016359,1.583425,0.518370,225.714286,0.254000,5.214623
